In [2]:
import os

from langchain_cohere import CohereEmbeddings , CohereRerank , ChatCohere
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import cohere
import qdrant_client
from qdrant_client.models import Batch
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams



In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
Qclient = QdrantClient(url="http://localhost:6333")
cohere_client = cohere.Client()
client = qdrant_client.QdrantClient(os.environ["COHERE_API_KEY"])

In [4]:
def cohereIndexing(client,collection,texts):
    client.upsert(
    collection_name=collection,
    points=Batch(
        ids=range(len(texts)),
        vectors=cohere_client.embed(
            model="embed-english-v3.0",  # New Embed v3 model
            input_type="search_document",  # Input type for indexing documents
            texts= [texts[i].page_content for i in range(len(texts))],
        ).embeddings,
        payloads = [{"Context{}".format(index): value} for index, value in enumerate([texts[i].page_content for i in range(len(texts))], start=1)],
    ),
    )

In [5]:
def cohereRetrival(collection,textList):
    cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
    client = qdrant_client.QdrantClient()
    result = client.search(
    collection_name=collection,
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=textList,
    ).embeddings[0],
    )
    return result

In [6]:
from langchain.prompts import ChatPromptTemplate
template = """You are a medical assistant that specializes in providing second opinions, diagnosing complex cases 
and suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan 
or second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment
plann is related to the patient's history.

Patient History : {patientHistory}

Medical Context : {context}

Task: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = prompt | llm

In [14]:
context = cohereRetrival("MedicalPapers",[question])

In [23]:
context[0].payload['Context813']

'4 \n 2.3 It should take place face to face in a confidential environment, with access for relatives and \nallied health professionals with a minimum time of 15 -20 minutes. Patients will often be \nmonths  or even years after injury and frustrated by the delay in diagnosis. It may be the first \ntime the diagnosis is made, but equally they may already have researched the diagnosis and \nmade  a decision on their preferred treatment. This decision may change foll owing the \nconsultation.  \n \n2.4 Past medical records from hospital and general practitioner including imaging (if any) should \nbe available.  \n \n2.5 The history of the injury, subsequent and current symptoms should be documented, followed \nby a clinical examination and re view of all imaging. Instrumented laxiometry may be helpful \nbut is not considered to be universally necessary.  \n 2.6 A management plan is made after discussion of both operative & non -operative options for'

In [ ]:
result = chain.invoke({"context":retrievedDocs[0].payload['Context722'] , "patientHistory":"","question":question})
to_markdown(result.content)

In [ ]:
result = chain.invoke({"context":context , "patientHistory":patientHistory,"question":question})
to_markdown(result.content)

In [29]:
evaluationLLm =  ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest")

In [26]:
def generate_queries(model,prompt, num_queries):
  query_generation_prompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
  query_generation_chain = query_generation_prompt | model
  return query_generation_chain.invoke({"prompt": prompt, "num_queries": num_queries}).content.split('\n')

In [33]:
question = 'My patient has had a full ACL reconstruction and repair following an ACL tear and a partial meniscus tear. To help him recover fully, what steps are required? How should we prevent a retear of the ACL ligament?'

In [34]:
queries = generate_queries(evaluationLLm,question,3)
retrievedDocs = cohereRetrival("MedicalPapers",queries[:1])

In [35]:
queries

['1. What are the essential stages of rehabilitation to optimize recovery after ACL reconstruction and meniscus repair?',
 '2. How can we effectively prevent a retear of the ACL ligament during the rehabilitation process?',
 '3. What are the specific precautions and adaptations required to minimize the risk of retear during functional activities post-rehabilitation?']

In [48]:
len(retrievedDocs)

10

In [ ]:
retrievedDocs[0].payload.values()

In [139]:
def cohereRetrival(collection,textList):
    cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
    client = qdrant_client.QdrantClient()
    result = client.search(
    collection_name=collection,
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=textList,
    ).embeddings[0],
    )
    return result

In [140]:
def getResponce(prompt,context = "",PatientHistory=""):
    result = chain.invoke({"context":context,"patientHistory":PatientHistory,"question":prompt})
    return result

In [141]:
import cohere

def ragFusion(prompt,collection= "MedicalPapers"):
    co = cohere.Client(os.environ["COHERE_API_KEY"])
    queryGenerationPrompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
    queryGenerationChain = queryGenerationPrompt | llm
    queries = queryGenerationChain.invoke({"prompt": prompt, "num_queries": 3}).content.split('\n')
    retrievedContent = []
    for query in queries:
        ret = cohereRetrival(collection,[query])
        for doc in ret:
            for key,value in doc.payload.items():
                value = value.replace(u'\xa0',u' ')
                value = value.replace(u'\t',u'  ')
                value = value.replace(u'\r',u'')
                value = value.replace(u'\n',u'      ')
                retrievedContent.append(value)
    retrievedContent = list(set(retrievedContent))
    result = co.rerank(model="rerank-english-v3.0", query=prompt, documents=retrievedContent, top_n=5, return_documents=True)
    context = ""
    for i in result.results:
        context += i.document.text
        print(i.document)
        context += "\n\n"

In [8]:
from langchain.vectorstores import Qdrant
from langchain_cohere import CohereEmbeddings
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

import qdrant_client


client = QdrantClient(url="http://localhost:6333")


embeddings = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=os.environ["COHERE_API_KEY"]
)

doc_store = Qdrant(
    client=client, collection_name="MedicalPapers", 
    embeddings=embeddings,
)

retriever = doc_store.as_retriever()

In [7]:
from langchain.prompts import ChatPromptTemplate
template = """You are a medical assistant that specializes in providing second opinions, diagnosing complex cases 
and suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan 
or second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment
plann is related to the patient's history.

Patient History : {patientHistory}

Medical Context : {context}

Task: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

retrival = RunnableParallel( "context": retriever,"question": RunnablePassthrough(), "Patient History": RunnablePassthrough())

output_parser = StrOutputParser()

chain = retrival | prompt | model | output_parser

SyntaxError: invalid syntax (1778620258.py, line 6)